In [ ]:
import sys, pygame
import tkinter as tk
from tkinter import filedialog
from operacion import Cell
from random import Random

key_patterns = {
    pygame.K_1: "1",
    pygame.K_2: "2",
    pygame.K_3: "3",
    pygame.K_4: "4",
    pygame.K_5: "5",
    pygame.K_6: "6", 
    pygame.K_7: "7",
    pygame.K_8: "8",
    pygame.K_9: "9",
    pygame.K_0: "10",
    pygame.K_u: "11"
}


import os
os.chdir(os.path.dirname(os.path.abspath(__file__)))

root = tk.Tk()
root.withdraw()

from operacion import operacion

size = width, height = 1000, 564 # ancho y alto de los pixeles en la ventana
bottom_bar_height = 64 #ancho de la barra de botones

black = (0,0,0)
white = (255, 255, 255)
gray = (92, 92, 92)

#Funcion para cambiar estado de las celdas
def mouse_click(world: operacion, mouse_x: int, mouse_y: int) -> None: 
    x = int(mouse_x / 25)
    y = int(mouse_y / 25)
    if world.read(x,y) == world.LIVE:
        world.write(x,y,world.DEAD)
    else:
        world.write(x, y, world.LIVE)
        #ENEMIGOS

#Termina enemigos
def main():
#inicializacion de la libreria
    pygame.init()
    #MODIFICADO
    enemy_pattern = [
[(0, 0), (1, 0),
(0, 1),
(1, 2), (2, 2)
],
[
 (0,0),(0,1),       
 (1,0),(1,1),
 (2,0)        
    ]]
    #TERMINA MODIFICACION
    pygame.font.init()
    world = operacion()

    #inicialización de la ventana
    screen = pygame.display.set_mode(size)
    pygame.display.set_caption("OPERACIÓN PATÓGENA")
    
    #iniciación de la imagen de fondo
    play = pygame.image.load("play.png")
    play = pygame.transform.scale(play, (50, 50))   
    playrect = play.get_rect()
    playrect = playrect.move(400, 505)

    pause = pygame.image.load("pause.png")
    pause = pygame.transform.scale(pause, (50, 50))
    pauserect = pause.get_rect()
    pauserect = pauserect.move(468, 505)

    clear = pygame.image.load("clear.png")
    clear = pygame.transform.scale(clear, (50, 50))
    clearrect = clear.get_rect()
    clearrect = clearrect.move(532, 505)

    load = pygame.image.load("load.png")
    load = pygame.transform.scale(load, (50, 50))
    loadrect = load.get_rect()
    loadrect = loadrect.move(596, 505)

    save = pygame.image.load("save.png")
    save = pygame.transform.scale(save, (50, 50))
    saverect = save.get_rect()
    saverect = saverect.move(660, 505 )

    myfont = pygame.font.SysFont('lucida Console', 30)

    running = False  
    game_over=False
    #MODIFICADO 
    good_cells = [[], [], [], []] 
    enemies = [[], [], [], []]

    enemy_spawn_timer = 0

    cell_put_on_it = False
    #TERMINA MODIFICACION
    #bucle de ejecución
    while 1:
        for event in pygame.event.get():
            cell_put_on_it = False
            
            def reset():
                world.reset()
                screen.fill(black)
                world.draw(screen)
                for cells_list in good_cells + enemies:
                    cells_list.clear()
                enemy_spawn_timer = 0

            #Evento cerrar ventana
            if event.type == pygame.QUIT:
                print("END...")
                sys.exit()
                
            #Evento de click del ratón
            if event.type == pygame.MOUSEBUTTONDOWN:
                x, y = pygame.mouse.get_pos()
                if y <= 500:
                    mouse_click(world, x, y)
                else:
                    if playrect.collidepoint(x,y):
                        running = True
                    if pauserect.collidepoint(x,y):
                        running = False
                    if clearrect.collidepoint(x,y):
                        reset()
                    if loadrect.collidepoint(x, y):
                        filepath = filedialog.askopenfilename(title="Cargar patrón", filetypes=[("Archivos de texto", "*.txt")])
                        if filepath:
                            world.load(filepath)
                    if saverect.collidepoint(x, y):
                        filepath = filedialog.asksaveasfilename(title="Guardar patrón", defaultextension=".txt", filetypes=[("Archivos de texto", "*.txt")])
                        if filepath:
                            world.save(filepath)

            if event.type == pygame.KEYDOWN: # presionar teclas para dibujar :DDD
                if event.key in key_patterns and not cell_put_on_it:

                    good_cells_len = 0
                    for sub_good_cells in good_cells:
                        good_cells_len += len(sub_good_cells)
                    
                    print(good_cells_len)
                
                    if good_cells_len < 25:

                        pattern = operacion.PATTERNS[key_patterns[event.key]].copy()

                        PATTERN_PRICE = [
                            25,
                            20,
                            15,
                            12,
                            10,
                            7,
                            4,
                            2,
                            1
                        ]

                        pattern_use = 0

                        for i in range(4):
                            for good_cell in good_cells[i]:
                                if good_cell.pattern_0 == key_patterns[event.key]:
                                    pattern_use += 1

                        if pattern_use < PATTERN_PRICE[len(pattern) - 1]:
                            mouse_y = pygame.mouse.get_pos()[1]
                            pixels_per_cell = width / operacion.WIDTH
                            cells_per_carril = 5
                            carril_height = cells_per_carril * pixels_per_cell

                            carril = min(int(mouse_y / carril_height), 3)

                            free_create_space = True
                            cs_delta = 4

                            for c in good_cells[carril] + enemies[carril]:
                                if c.x <= cs_delta:
                                    free_create_space = False
                                    break
                                

                            if free_create_space:
                                x = 0
                                if not running:
                                    x = 1
                                good_cell = Cell(x, carril * cells_per_carril + 1, pattern, True)
                                good_cells[carril].append(good_cell)
                                good_cell.draw(world)
                                cell_put_on_it = True


        #actualizacion AC
        if running and not game_over:
            world.update()
            pygame.time.delay(500) #delay de 500ms entre actualización de celdas
        # ENEMIGOS
            for i in range(4):
                for enemy in enemies[i]:
                    cell_i = enemy.detect_cells(good_cells[i], enemies[i])
                    if enemy.next_move == Cell.MOVE:
                        enemy.erase(world)
                        enemy.move_left()
                        if enemy.x <= 0:
                            running = False
                            game_over = True
                            break

                    elif enemy.next_move == Cell.FIGHT:
                        good_cell = good_cells[i][cell_i]
                        enemy.erase(world)
                        to_erase = Cell.cells_fight(good_cell, enemy)
                        if enemy.pattern == []:
                            enemies[i].remove(enemy)
                        if good_cell.pattern == []:
                            good_cells[i].pop(cell_i)
                        for x,y in to_erase:
                            world.write(x, y, world.DEAD)


            for i in range(4):
                enemies[i] = [enemy for enemy in enemies[i] if not enemy.is_offscreen()]
                #TERMINA EL JUEGO
            for i in range(4):
                for enemy in enemies [i]:
                    if enemy.x <=0:
                        running =False
                        game_over=True
                        break
                else:
                    continue
                break
            for sub_enemies in enemies:
                for enemy in sub_enemies:
                    enemy.draw(world)
               
            enemy_spawn_timer += 1
            if enemy_spawn_timer >= 5:  # cada 5 ciclos
                enemy_spawn_timer = 0
                from random import choice
                pattern = choice(list(operacion.PATTERNS.values())).copy()

                carril = Random().randint(0, 3)
                enemies[carril].append(Cell(38, carril * 5 + 1, pattern, False))  # 38*25 = 950px
    
            #CELULAS BUENAS
        
            for i in range(4):
                for good_cell in good_cells[i]:
                    if good_cell.next_move == Cell.FIGHT:
                        good_cell.erase(world)
                        continue
                    good_cell.detect_cells(good_cells[i], enemies[i])
                    if good_cell.next_move == Cell.MOVE:
                        good_cell.erase(world)
                        good_cell.move_right()
                    if good_cell.next_move == Cell.WAIT:
                        good_cell.next_move = Cell.MOVE

            for i in range(4):
                good_cells[i] = [good_cell for good_cell in good_cells[i] if not good_cell.is_offscreen()]
            
            for sub_good_cells in good_cells:
                for good_cell in sub_good_cells:
                    good_cell.draw(world)

        # refrescar
        screen.fill(black)
        world.draw(screen)

        # Dibujar los 4 carriles horizontales
        for i in range(4):
            carril_y = i * 125  # Separación vertical entre carriles (500px / 4)
            pygame.draw.rect(screen, gray, pygame.Rect(0, carril_y, 1000, 5))
            create_space_width = 2
            pygame.draw.line(screen, (0, 0, 255), (25, carril_y + 25), (100, carril_y + 25), create_space_width)
            pygame.draw.line(screen, (0, 0, 255), (25, carril_y + 25), (25, carril_y + 100), create_space_width)
            pygame.draw.line(screen, (0, 0, 255), (100, carril_y + 100), (100, carril_y + 25), create_space_width)
            pygame.draw.line(screen, (0, 0, 255), (100, carril_y + 100), (25, carril_y + 100), create_space_width)
        screen.blit(play, playrect)
        screen.blit(pause, pauserect)
        screen.blit(clear, clearrect)
        screen.blit(load, loadrect)
        screen.blit(save, saverect)

        textsurface = myfont.render("RUNNING" if running else "PAUSE", True, white)
        if game_over:
         game_over_font = pygame.font.SysFont('lucida Console', 60)
         game_over_surface = game_over_font.render("GAME OVER", True, (255, 0, 0))
         screen.fill(black)
         screen.blit(game_over_surface, (width // 2 - 160, height // 2 - 30))
         world.draw(screen)
         pygame.display.flip()
         pygame.time.wait(5000)
         game_over = False
         reset()

        screen.blit(textsurface, (100, 516))
        screen.blit(
            myfont.render(str(world.iterations), True, white), (750,516))
        pygame.display.flip()

if __name__ == "__main__":
    main()


In [ ]:
import pygame

class Cell:

    QUIET = 0
    MOVE = 1
    FIGHT = 2
    WAIT = 3

    DAMAGE_STAGE_0 = 1_000
    DAMAGE_STAGE_1 = 1_001
    DAMAGE_STAGE_2 = 1_002
    DAMAGE_STAGE_3 = 1_003
    BAD_ADD = 1_000

    DAMAGE_STAGE_0_COLOR = (255, 255, 255)
    DAMAGE_STAGE_0_COLOR_BAD = (122, 106, 106)
    DAMAGE_STAGE_1_COLOR = (225, 170, 170)
    DAMAGE_STAGE_1_COLOR_BAD = (95, 90, 115)
    DAMAGE_STAGE_2_COLOR = (190, 85, 85)
    DAMAGE_STAGE_2_COLOR_BAD = (65, 70, 120)
    DAMAGE_STAGE_3_COLOR = (158, 0, 0)
    DAMAGE_STAGE_3_COLOR_BAD = (40, 50, 130)

    def __init__(self, x, y, pattern, good):
        self.x = x
        self.y = y
        self.good = good
        self.pattern = pattern.copy()  # Lista de tuplas (dx, dy)
        self.pattern_0 = ""
        for key in operacion.PATTERNS.keys():
            if operacion.PATTERNS.get(key) == pattern:
                self.pattern_0 = key
        print(self.pattern_0)
        self.next_move = self.MOVE
        self.damage_bitmap = []
        for xy in pattern:
            self.damage_bitmap.append(self.DAMAGE_STAGE_0)
        

    def draw(self, world):
        counter = 0
        for dx, dy in self.pattern:   
            value = self.damage_bitmap[counter]
            if not self.good:
                value += self.BAD_ADD
            world.write(self.x + dx, self.y + dy, value)
            counter += 1

    def erase(self, world):
        for dx, dy in self.pattern:
            world.write(self.x + dx, self.y + dy, world.DEAD)

    def move_left(self):
        self.x -= 1

    def move_right(self):
        self.x += 1

    def detect_cells(self, good_cells, enemies) -> int:
        if self.good:
            for i in range (len(good_cells)):
                cell = good_cells[i]
                cell_x = cell.x
                x_diff = cell_x - self.x - max(dx for dx,dy in self.pattern)
                if x_diff == 0:
                    continue
                elif x_diff <= 2 and x_diff > 0:
                    self.next_move = Cell.QUIET
                    return 0
                elif self.next_move != Cell.WAIT:
                    self.next_move = Cell.MOVE
            return 0
        else:
            cells = good_cells + enemies
            for i in range(len(cells)):
                cell = cells[i]
                cell_x = cell.x + max(dx for dx, dy in cell.pattern)
                cell_good = cell.good
                self_x = self.x
                self_pattern = self.pattern
                x_diff = self.x - cell_x
                if x_diff == 2:
                    self.next_move = Cell.QUIET
                    return 0
                elif cell in good_cells:
                    if x_diff == 1:
                        self.next_move = Cell.FIGHT
                        cell.next_move = Cell.FIGHT
                        return i
            self.next_move = Cell.MOVE
            return 0

    def is_offscreen(self):
        return self.x == 0  or self.x + max(dx for dx,dy in self.pattern) == operacion.WIDTH

    def cells_fight(good_cell, bad_cell) -> any:
        
        good_cell_warriors = 0
        good_cell_max_x = max(dx for dx,dy in good_cell.pattern)
        for i in range(len(good_cell.pattern) -1, -1, -1):
            if (good_cell.pattern[i][0] < good_cell_max_x):
                break
            good_cell_warriors += 1
        
        bad_cell_warriors = 0
        for i in range(len(bad_cell.pattern)):
            if bad_cell.pattern[i][0] > 0:
                break
            bad_cell_warriors += 1

        to_erase = []

        if good_cell_warriors > bad_cell_warriors:
            
            for w in range(bad_cell_warriors):
                point = bad_cell.pattern.pop(0)
                bad_cell.damage_bitmap.pop(0)
                to_erase.append((bad_cell.x + point[0], bad_cell.y + point[1]))
            for p in range(len(bad_cell.pattern)):
                point = bad_cell.pattern[p]
                x = point[0]
                y = point[1]
                bad_cell.pattern.pop(p)
                bad_cell.pattern.insert(p, (x - 1, y))
            bad_cell.x += 1

            damage_counter = bad_cell_warriors
            i = len(good_cell.pattern) - 1
            while damage_counter > 0:
                iDamage = good_cell.damage_bitmap[i]
                if iDamage + damage_counter <= Cell.DAMAGE_STAGE_3:
                    good_cell.damage_bitmap[i] += damage_counter
                    damage_counter = 0
                else:
                    damage_counter -= Cell.DAMAGE_STAGE_3 + 1 - iDamage
                    point = good_cell.pattern[i]
                    to_erase.append((good_cell.x + point[0], good_cell.y + point[1]))
                    good_cell.pattern.pop(i)
                    good_cell.damage_bitmap.pop(i)
                    i-=1
            
            good_cell.next_move = Cell.WAIT
        
        if bad_cell_warriors > good_cell_warriors:

            for w in range(good_cell_warriors):
                point = good_cell.pattern.pop()
                good_cell.damage_bitmap.pop()
                to_erase.append((good_cell.x + point[0], good_cell.y + point[1]))
            
            damage_counter = good_cell_warriors
            i = 0
            while damage_counter > 0:
                iDamage = bad_cell.damage_bitmap[i]
                if iDamage + damage_counter <= Cell.DAMAGE_STAGE_3:
                    bad_cell.damage_bitmap[i] += damage_counter
                    damage_counter = 0
                else:
                    damage_counter -= Cell.DAMAGE_STAGE_3 + 1 - iDamage
                    point = bad_cell.pattern[i]
                    to_erase.append((bad_cell.x + point[0], bad_cell.y + point[1]))
                    bad_cell.pattern.pop(i)
                    bad_cell.damage_bitmap.pop(i)
                    i-=1
            
            bad_cell.next_move = Cell.QUIET
            if good_cell.pattern != []:
                good_cell.next_move = Cell.WAIT
        
        if good_cell_warriors == bad_cell_warriors:

            good_cell_damage = good_cell.damage_bitmap[-1]
            bad_cell_damage = bad_cell.damage_bitmap[0]

            if good_cell_damage == bad_cell_damage:
                for w in range(good_cell_warriors):
                    good_point = good_cell.pattern.pop()
                    good_cell.damage_bitmap.pop()
                    bad_point = bad_cell.pattern.pop(0)
                    bad_cell.damage_bitmap.pop(0)
                    to_erase.append((good_point[0] + good_cell.x, good_point[1] + good_cell.y))
                    to_erase.append((bad_point[0] + bad_cell.x, bad_point[1] + bad_cell.y))
                for p in range(len(bad_cell.pattern)):
                    point = bad_cell.pattern[p]
                    x = point[0]
                    y = point[1]
                    bad_cell.pattern.pop(p)
                    bad_cell.pattern.insert(p, (x - 1, y))
                bad_cell.x += 1
                
                if good_cell.pattern != []:
                    good_cell.next_move = Cell.WAIT
            elif good_cell_damage < bad_cell_damage:
                for w in range(bad_cell_warriors):
                    point = bad_cell.pattern.pop(0)
                    bad_cell.damage_bitmap.pop(0)
                    to_erase.append((bad_cell.x + point[0], bad_cell.y + point[1]))
                for p in range(len(bad_cell.pattern)):
                    point = bad_cell.pattern[p]
                    x = point[0]
                    y = point[1]
                    bad_cell.pattern.pop(p)
                    bad_cell.pattern.insert(p, (x - 1, y))
                bad_cell.x += 1
                for w in range(good_cell_warriors - 1):
                    point = good_cell.pattern.pop()
                    good_cell.damage_bitmap.pop()
                    to_erase.append((good_cell.x + point[0], good_cell.y + point[1]))
                good_cell.damage_bitmap[-1] += Cell.DAMAGE_STAGE_3 + 1 - bad_cell_damage
                
                good_cell.next_move = Cell.WAIT
            elif good_cell_damage > bad_cell_damage:
                for w in range(good_cell_warriors):
                    point = good_cell.pattern.pop()
                    good_cell.damage_bitmap.pop()
                    to_erase.append((good_cell.x + point[0], good_cell.y + point[1]))
                for w in range(bad_cell_warriors - 1):
                    point = bad_cell.pattern.pop(0)
                    bad_cell.damage_bitmap.pop(0)
                    to_erase.append((bad_cell.x + point[0], bad_cell.y + point[1]))
                bad_cell.damage_bitmap[0] += Cell.DAMAGE_STAGE_3 + 1 - good_cell_damage
                
                if good_cell.pattern != []:
                    good_cell.next_move = Cell.WAIT
            
            bad_cell.next_move = Cell.QUIET

        
        return to_erase

class operacion:
    PATTERNS = {
    "1": [(0,1)],
    "2": [(0,0), (0,1), (0,2), (1,1)],
    "3": [(0,1), (1,1)],
    "4": [(0,0), (0,1), (0,2)],
    "5": [(0,2), (1,1), (2,0)],
    "6": [(0,0), (1,1), (2,2)],
    "7": [(0, 0), (0, 1), (1, 0), (1, 1)],
    "8": [(0,1), (1,0)],
    "9": [(0,0), (0,1), (0,2), (1,2)],
    "10": [(0,0), (0,1), (0,2), (1,2), (1,1)],
    "11": [(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (2, 0), (2, 1), (2, 2)]
    }

    WIDTH = 40 # ANCHO DE LAS CELDAS
    HEIGHT = 20 # ALTO DE LAS CELDAS
    LIVE = 1 # VALOR CELDA VIVA
    DEAD = 0 # VALOR CELDA MUERTA



    __world = [] # lista donde cada valor corresponde al estado de una celda, ESTADO PRINCIPAL
    __next = [] # lista de respaldo, cuando hacemos el paso de una iteración a la siguiente

    __born = [] #lista de numeros de celdas vecinas activas para que una celda se active
    __alive = [] # lista de numero de celdas vecinas activas para que una celda se mantenga activa

    __iterations = 0 #contador de iteraciones
    
    #constructor
    #formato basico del juego de la vida de conway

    def insert_pattern(self, pattern_name: str, x: int, y: int) -> None:
     if pattern_name in self.PATTERNS:
        for dx, dy in self.PATTERNS[pattern_name]:
            self.write(x + dx, y + dy, self.LIVE)



    def __init__(self, pattern: str = "23/3"):  # Pattern por defecto 23/3 
        self.reset()
        self.__alive = [int(v) for v in pattern.split("/")[0]] #2.3
        self.__born = [int(v) for v in pattern.split("/")[1]] #3

    @property #propiedades
    def iterations(self) -> int:
        return self.__iterations #devuelve el numero de iteraciones procesadas

    @property
    def livecells(self) -> int:
        return self.__world.count(1) #devuelve el numero de celdas activas en el espacio

    def reset (self): #inicializa las listas con ceros
        self.__iterations = 0
        self.__world = [0] * (self.WIDTH * self.HEIGHT)
        self.__next = [0] * (self.WIDTH * self.HEIGHT)

    def read(self, x: int, y: int) -> int:
        # permite obtener el estado de una celda con sus cordenadas x y y, accediendo a la lista world
        # retorna el valor de la celda activa o inactiva (esos son los estados que tiene el juego de la vida)
        if x >= self.WIDTH:
            x-= self.WIDTH

        elif x < 0:
            x+= self.WIDTH

        if y >= self.HEIGHT:
            y -= self.HEIGHT

        elif y < 0:
            y += self.HEIGHT

        return self.__world[(y * self.WIDTH) + x]

    def write(self, x: int, y: int, value: int) -> None: #establece el valor de una celda teniendo en cuenta sus coordenadas. 
        self.__world[(y * self.WIDTH) + x] = value

    def update(self) -> None:
        self.__iterations += 1  # Incrementa el contador de iteraciones

            
    def draw(self, context: pygame.Surface) -> None:
        cell_size = 25  # cuadrado: 25 x 25

        for y in range(self.HEIGHT):
            for x in range(self.WIDTH):
                current = self.read(x, y)
                rect = (x * cell_size, y * cell_size, cell_size, cell_size)
                color = ()
                if current == Cell.DAMAGE_STAGE_0:
                    color = Cell.DAMAGE_STAGE_0_COLOR
                elif current == Cell.DAMAGE_STAGE_1:
                    color = Cell.DAMAGE_STAGE_1_COLOR
                elif current == Cell.DAMAGE_STAGE_2:
                    color = Cell.DAMAGE_STAGE_2_COLOR
                elif current == Cell.DAMAGE_STAGE_3:
                    color = Cell.DAMAGE_STAGE_3_COLOR
                elif current == Cell.DAMAGE_STAGE_0 + Cell.BAD_ADD:
                    color = Cell.DAMAGE_STAGE_0_COLOR_BAD
                elif current == Cell.DAMAGE_STAGE_1 + Cell.BAD_ADD:
                    color = Cell.DAMAGE_STAGE_1_COLOR_BAD
                elif current == Cell.DAMAGE_STAGE_2 + Cell.BAD_ADD:
                    color = Cell.DAMAGE_STAGE_2_COLOR_BAD
                elif current == Cell.DAMAGE_STAGE_3 + Cell.BAD_ADD:
                    color = Cell.DAMAGE_STAGE_3_COLOR_BAD

                if color != ():
                    pygame.draw.rect(surface=context, color=color, rect=rect)
                else:
                    pygame.draw.rect(surface=context, color=(64, 64, 64), rect=rect, width=1)

    #Funciones para guardar y cargar el estado del juego
    def save(self, filename:str) -> None:
        with open(filename, mode="w", encoding="utf-8") as fp:
            fp.write(str(self.__world))
    
    def load(self, filename:str) -> None:
        with open(filename, mode="r", encoding="utf-8") as fp:
            data = fp.read()[1:-1]
            self.__world = [int(v) for v in data.split(",")]

